In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

### Базовые URL для парсинга

In [72]:
base_url = "https://lifehacker.ru/"
base_url_tech_page = "https://lifehacker.ru/topics/technology/?page={}"

### Парсинг ссылок на статьи с главной страницы

In [73]:
parsed_urls = []
pages_to_parse = 10

for page_num in range(1, pages_to_parse + 1):
    url = base_url_tech_page.format(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-small-article-card__link') # получаем ссылку на статью из карточки
    parsed_urls += [item.get('href') for item in raw_items]

### Парсинг титульника и текста с каждой статьи

In [75]:
result = []

for url in tqdm(parsed_urls):
    article = {}

    article_url = f'{base_url}{url}' # подставляем часть ссылки, ведущую к материалу
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'lxml')

    # Безопасное извлечение заголовка
    title_tag = soup.find('h1', class_='article-card__title')
    article['title'] = title_tag.text.strip() if title_tag else None

    # Безопасное извлечение текста
    content_tag = soup.find('article', class_='single-article__post-content')
    article['text'] = content_tag.text.strip() if content_tag else None

    result.append(article)

100%|██████████| 300/300 [09:21<00:00,  1.87s/it]


### Сделаем красивый датафрейм

In [76]:
pd.set_option('display.max_colwidth', 400)

In [77]:
data = pd.DataFrame(result)
data.head()

,title,text
0,Промпт дня: «напиши и выполни» — чтобы чат-бот задал вопрос сам себе,"На Лайфхакере новая рубрика — «Промпт дня». В ней мы будем давать советы по работе с чат-ботами, которые помогут улучшить выдачу и придумать новые способы внедрить нейросети в свою жизнь с пользой. Чат-ботов много, но мы стараемся делать рекомендации максимально универсальными: чтобы сработало и в ChatGPT, и в DeepSeek, и в Le Chat.И вполне символично, что мы начинаем новую рубрику с промпта, ..."
1,Вышла Midjourney V7 — с режимом черновика и голосовым вводом,"В блоге нейросети Midjourney появился анонс версии V7, которая гораздо лучше справляется с текстовыми подсказками, понимает голосовые и позволяет получать более качественные изображения. По словам разработчиков, Midjourney V7 сильно превосходит прошлую версию по части работы с красивыми текстурами, руками людей, кожей и согласованностью всех деталей изображения. Кроме того, это первая модель, ..."
2,"Apple запатентовала полностью стеклянные iPhone, Apple Watch и Mac Pro","Apple до сих пор одержима идеей полностью стеклянного iPhone, пишет MacRumors. Об этом свидетельствует последний патент компании в США, в котором подробно описан такой смартфон. Более того, подобный корпус может использоваться и в других устройствах, включая Apple Watch и даже Mac Pro.В документе рассказывается о девайсах, в которых все внешние поверхности (передняя, задняя и боковые) выполнен..."
3,Представлен «вечный блокнот» HEYfold — на нём можно бесконечно писать и стирать текст,"Стартап HEYii представил «вечный блокнот» HEYfold, который позволяет стирать записи салфеткой и снова писать. Разработчики считают, что он может занять золотую середину между традиционными ежедневниками и цифровыми приложениями.Изображение: HEYiiЭто ультратонкий гаджет толщиной всего 4 миллиметра, внешне напоминающий чистый лист бумаги. Для письма есть несколько разных ручек, а удалять записи ..."
4,Складной iPhone Fold получит внешний экран размером с iPhone 13 mini — для любителей компактности,"Продажи iPhone 13 mini оказались ниже ожиданий Apple, однако этот небольшой смартфон всё же нашёл свою аудиторию. Несмотря на то, что компания решила отказаться от выпуска модели, её особенности будут учтены при разработке складного iPhone Fold, который должен выйти уже в следующем году. Так купертинцы стремятся удовлетворить запросы поклонников компактности.По данным инсайдера Мин-Чи Куо, вне..."
